In [1]:
import os
import json
import shutil
import numpy as np
import parmed as pmd

import paprika.io as io
import paprika.restraints as restraints
import paprika.restraints.amber as amber
import paprika.restraints.utils as utils
import paprika.restraints.plumed as plumed
import paprika.restraints.colvars as colvars

### Initialize

In [2]:
base_name = "cb6-but-dum"

### Define the number of windows

In [3]:
attach_string = "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
attach_fractions = [float(i) / 100 for i in attach_string.split()]

initial_distance = 6.0
pull_distances = np.arange(0.0 + initial_distance, 18.0 + initial_distance, 1.0)

release_fractions = attach_fractions[::-1]

windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
print(f"There are {windows} windows in this attach-pull-release calculation.")

There are [15, 18, 15] windows in this attach-pull-release calculation.


### Define anchor atoms

In [4]:
# Guest atoms
G1 = ":BUT@C"
G2 = ":BUT@C3"

# Host atoms
H1 = ":CB6@C"
H2 = ":CB6@C31"
H3 = ":CB6@C18"

# Dummy atoms
D1 = ":DM1"
D2 = ":DM2"
D3 = ":DM3"

### Load vacuum structure

In [5]:
structure = pmd.load_file(
    f"complex/{base_name}.prmtop",
    f"complex/{base_name}.rst7",
    structure = True,
) 

### Host Static Restraints

In [6]:
static_restraints = []

In [7]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D1, H1],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 5.0,
                                    amber_index=True)

static_restraints.append(r)

In [8]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D2, D1, H1],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=True)

static_restraints.append(r)

In [9]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D3, D2, D1, H1],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=True)

static_restraints.append(r)

In [10]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D1, H1, H2],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=True)

static_restraints.append(r)

In [11]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D2, D1, H1, H2],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=True)

static_restraints.append(r)

In [12]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D1, H1, H2, H3],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=True)

static_restraints.append(r)

### Host Conformational Restraints (Jack restraints)

In [13]:
host_restraints = []
r0 = 11.5
kr = 15.0
bonds = [
    [':CB6@N1', ':CB6@N13'],
    [':CB6@N3', ':CB6@N15'],
    [':CB6@N5', ':CB6@N17'],
    [':CB6@N7', ':CB6@N19'],
    [':CB6@N9', ':CB6@N21'],
    [':CB6@N11', ':CB6@N23'],
    [':CB6@N2', ':CB6@N14'],
    [':CB6@N', ':CB6@N12'],
    [':CB6@N10', ':CB6@N22'],
    [':CB6@N8', ':CB6@N20'],
    [':CB6@N6', ':CB6@N18'],
    [':CB6@N4', ':CB6@N16'],
]

In [14]:
for bond in bonds:
    r = restraints.DAT_restraint()
    r.mask1 = bond[0]
    r.mask2 = bond[1]
    r.topology = structure
    r.auto_apr = True
    r.continuous_apr = True
    r.amber_index = True

    r.attach["target"] = r0                        # Angstroms
    r.attach["fraction_list"] = attach_fractions
    r.attach["fc_final"] = kr                      # kcal/mol/Angstroms**2

    r.pull["target_final"] = r0                    # Angstroms
    r.pull["num_windows"] = windows[1]
    
    r.release["target"] = r0                       # Angstroms
    r.release["fraction_list"] = release_fractions
    r.release["fc_final"] = kr                     # kcal/mol/Angstroms**2

    r.initialize()
    
    host_restraints.append(r)

### Guest translational and rotational restraints

In [15]:
guest_restraints = []

In [16]:
r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = True

r.attach["target"] = pull_distances[0]          # Angstroms
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 5.0                      # kcal/mol/Angstroms**2

r.pull["target_final"] = 24.0                   # Angstroms
r.pull["num_windows"] = windows[1]

r.release["target"] = pull_distances[-1]        # Angstroms
r.release["fraction_list"] = [1.0] * windows[2]
r.release["fc_final"] = 5.0                     # kcal/mol/Angstroms**2

r.initialize()
guest_restraints.append(r)

In [17]:
r = restraints.DAT_restraint()
r.mask1 = D2
r.mask2 = D1
r.mask3 = G1
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = True

r.attach["target"] = 180.0                      # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 100.0                    # kcal/mol/radian**2

r.pull["target_final"] = 180.0                  # Degrees
r.pull["num_windows"] = windows[1]

r.release["target"] = 180.0                     # Degrees
r.release["fraction_list"] = [1.0] * windows[2]
r.release["fc_final"] = 100.0                   # kcal/mol/radian**2

r.initialize()
guest_restraints.append(r)

In [18]:
r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.mask3 = G2
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = True

r.attach["target"] = 180.0                      # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 100.0                    # kcal/mol/radian**2

r.pull["target_final"] = 180.0                  # Degrees
r.pull["num_windows"] = windows[1]

r.release["target"] = 180.0                     # Degrees
r.release["fraction_list"] = [1.0] * windows[2]
r.release["fc_final"] = 100.0                   # kcal/mol/radian**2

r.initialize()
guest_restraints.append(r)

### Create APR windows and save restraints to Json file

In [19]:
window_list = restraints.restraints.create_window_list(guest_restraints)

apr_windows = {'attach': [], 'pull': [], 'release': []}
for window in window_list:
    if window[0] == "a":
        apr_windows["attach"].append(window)
    if window[0] == "p":
        apr_windows["pull"].append(window)
    if window[0] == "r":
        apr_windows["release"].append(window)
        
with open("windows/windows.json", "w") as f:
    dumped = json.dumps(apr_windows, cls=io.NumpyEncoder)
    f.write(dumped)

for window in window_list:
    if os.path.isdir(f"windows/{window}"):
        shutil.rmtree(f"windows/{window}")
    os.makedirs(f"windows/{window}")
    
host_guest_restraints = (static_restraints + host_restraints + guest_restraints)
io.save_restraints(host_guest_restraints, filepath="windows/restraints.json")

### Translate guest molecule

In [20]:
for window in window_list:
    if window[0] == "a":
        shutil.copy(f"complex/{base_name}.prmtop", f"windows/{window}/{base_name}.prmtop")
        shutil.copy(f"complex/{base_name}.rst7",   f"windows/{window}/{base_name}.rst7")

    elif window[0] == "p":
        structure = pmd.load_file(f"complex/{base_name}.prmtop", f"complex/{base_name}.rst7", structure = True)
        target_difference = guest_restraints[0].phase['pull']['targets'][int(window[1:])] - guest_restraints[0].pull['target_initial']
        print(f"In window {window} we will translate the guest {target_difference:0.1f} Angstroms.")
        
        for atom in structure.atoms:
            if atom.residue.name == "BUT":
                atom.xz += target_difference
                
        structure.save(f"windows/{window}/{base_name}.prmtop", overwrite=True)
        structure.save(f"windows/{window}/{base_name}.rst7", overwrite=True)

    elif window[0] == "r":
        last_pull_window = f"windows/p0{len(pull_distances)-1}"
        shutil.copy(f"{os.path.join(last_pull_window, f'{base_name}.prmtop')}", 
                    f"windows/{window}/{base_name}.prmtop")
        shutil.copy(f"{os.path.join(last_pull_window, f'{base_name}.rst7')}", 
                    f"windows/{window}/{base_name}.rst7")

In window p000 we will translate the guest 0.0 Angstroms.
In window p001 we will translate the guest 1.1 Angstroms.
In window p002 we will translate the guest 2.1 Angstroms.
In window p003 we will translate the guest 3.2 Angstroms.
In window p004 we will translate the guest 4.2 Angstroms.
In window p005 we will translate the guest 5.3 Angstroms.
In window p006 we will translate the guest 6.4 Angstroms.
In window p007 we will translate the guest 7.4 Angstroms.
In window p008 we will translate the guest 8.5 Angstroms.
In window p009 we will translate the guest 9.5 Angstroms.
In window p010 we will translate the guest 10.6 Angstroms.
In window p011 we will translate the guest 11.6 Angstroms.
In window p012 we will translate the guest 12.7 Angstroms.
In window p013 we will translate the guest 13.8 Angstroms.
In window p014 we will translate the guest 14.8 Angstroms.
In window p015 we will translate the guest 15.9 Angstroms.
In window p016 we will translate the guest 16.9 Angstroms.
In wind

### Write Amber NMR-style restraints

In [21]:
for window in window_list:
    with open(f"windows/{window}/disang.rest", "w") as file:
        restraints = utils.parse_restraints(
            static=static_restraints,
            host=host_restraints,
            guest=guest_restraints,
            list_type='tuple',
        )
        for restraint in restraints:
            string = amber.amber_restraint_line(restraint, window)
            if string is not None:
                file.write(string)

### Write PLUMED-based restraints

In [22]:
for window in window_list:
    with open(f"windows/{window}/plumed.dat", "w") as file:
        restraints = utils.parse_restraints(
            static=static_restraints,
            host=host_restraints,
            guest=guest_restraints,
            list_type='dict',
        )
        plumed.plumed_colvar_file(file, restraints, window, legacy_k=True)
    
    # Add dummy atoms
    with open(f"windows/{window}/plumed.dat", "a") as file:
        
        structure = pmd.load_file(
            f"windows/{window}/{base_name}.prmtop", 
            f"windows/{window}/{base_name}.rst7", 
            structure=True
        )
        plumed.add_dummy_to_plumed(structure, plumed_file=f"windows/{window}/plumed.dat")

### Write COLVAR Module-based restraints

In [23]:
for window in window_list:
    with open(f"windows/{window}/colvars.tcl", "w") as file:
        restraints = utils.parse_restraints(
            static=static_restraints,
            host=host_restraints,
            guest=guest_restraints,
            list_type='dict',
        )
        colvars.colvar_module_file(file, restraints, window, legacy_k=True)
    
    with open(f"windows/{window}/colvars.tcl", "a") as file:
        # Add dummy atoms
        structure = pmd.load_file(
            f"windows/{window}/{base_name}.prmtop", 
            f"windows/{window}/{base_name}.rst7", 
            structure=True
        )
        colvars.add_dummy_to_colvar(structure, colvar_file=f"windows/{window}/colvars.tcl")